# Initialization

In [1]:
# -*- coding: utf-8 -*-
"""
Fast mesh->graph pipeline (ProcessPool + caching + timing logger)

Usage:
  python build_graphs.py
  # 또는 인자 조정:
  python build_graphs.py --data_root "D:/ahmed_data" --target_field "static(p)_coeffMean" --run_range 1 30 --edge_mode auto --no_cache
"""

import os
# ---- 내부 OpenMP/BLAS 스레드 폭주 방지 (N 프로세스 × 1 스레드) ----
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

import argparse
import json
import hashlib
import time
import re
import difflib
import random
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed

import numpy as np
import torch
from tqdm.auto import tqdm
# from scipy.spatial import cKDTree


c:\Users\Junyoung Kim\.conda\envs\pyg5090\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# ---- PyG가 있으면 Data 사용, 없으면 최소 대체 ----
try:
    from torch_geometric.data import Data as GeoData
    print("Using torch_geometric Data class")
except Exception:
    class GeoData:
        def __init__(self, x, edge_index, y, y_graph):
            self.x = x
            self.edge_index = edge_index
            self.y = y
            self.y_graph = y_graph

Using torch_geometric Data class


# Utilities

In [3]:
# ----------------------------
# 기본 옵션(필요 시 CLI로 덮어쓰기)
# ----------------------------
KNN_K = 8
STRICT_BOUNDARY_ONLY = True
AUTO_GUESS_TARGET = True
NORMALIZE_TARGET = False
USE_NORMALS = False
INCLUDE_TARGET_IN_X = False
NORMALIZE_X = False
ALLOW_DTYPE = (np.float32, np.float64)
MAX_FEATURE_DIM = 16

EDGE_MODE = "auto"          # auto | face | knn
DO_CELL_TO_POINT = True
DO_COMPUTE_NORMALS = False
RUN_RANGE = (1, 101)         # None 이면 전체

CACHE_ENABLE = True
CACHE_PATH = Path("graphs_cache.pt")
CACHE_META = Path("graphs_cache.meta.json")


In [4]:
import re, numpy as np
from pathlib import Path
import pandas as pd
import json, os, tempfile

def _normpath(p):  # 경로 표준화
    try:
        return str(Path(p).resolve())
    except Exception:
        return str(Path(p))


def extract_run_id_from_mesh(path_str, pattern=r"run[_\-]?(\d+)"):
    p = Path(path_str)

    # 1) 모든 부모 폴더에서 run_* 찾기
    for parent in p.parents:
        m = re.search(pattern, parent.name.strip(), flags=re.IGNORECASE)
        if m:
            return m.group(1)

    # 2) 경로의 모든 파츠에서도 시도 (드문 케이스 대비)
    for part in p.parts:
        m = re.search(pattern, str(part).strip(), flags=re.IGNORECASE)
        if m:
            return m.group(1)

    # 3) 마지막으로 파일명(확장자 제외)에서 시도
    m = re.search(pattern, p.stem.strip(), flags=re.IGNORECASE)
    if m:
        return m.group(1)

    return None

def _extract_run_id(path_str: str, pattern=r"run[_\-]?(\d+)"):
    m = re.search(pattern, path_str, flags=re.IGNORECASE)
    return m.group(1) if m else None

def _build_csv_index(csv_dir, csv_glob="*.csv", run_id_regex=(r"run[_\-]?(\d+)", r"_(\d+)$", r"(\d+)$")):
    """
    csv_dir 안의 CSV들을 훑어서
    - 파일명에서 run_id 정규식으로 추출
    - 실패하면 CSV 내용의 열(run_id, Run, case_id 등)에서 시도
    - 그래도 실패하면 파일 전체경로를 key로 저장 (후보군)
    return: dict(run_id -> Path), dict('__unmatched__' -> [Path, ...])
    """
    csv_dir = Path(csv_dir)
    idx = {}
    unmatched = []
    for p in sorted(csv_dir.rglob(csv_glob)):
        rid = _extract_run_id(p.stem, run_id_regex)
        if rid is None:
            # 내용에서 찾기 시도 (있으면 사용)
            try:
                df = pd.read_csv(p, nrows=5)  # 가볍게 헤더만
                cand_cols = [c for c in df.columns if str(c).lower() in ("run_id","run","case_id","case","id")]
                if cand_cols:
                    val = str(df[cand_cols[0]].iloc[0])
                    if isinstance(val, str) and val.strip():
                        rid = re.sub(r"\D+", "", val) or val  # 숫자만 뽑아보거나 원문
            except Exception:
                pass
        if rid is None:
            unmatched.append(p)
        else:
            idx[str(rid)] = p
    return idx, {"__unmatched__": unmatched}

def _find_csv_for_mesh(mesh_path: str, idx: dict, fallbacks: dict, run_id_regex=(r"run[_\-]?(\d+)", r"_(\d+)$", r"(\d+)$")):
    """
    1순위: 메쉬 파일명에서 run_id 추출 → idx에서 찾기
    2순위: 메쉬와 같은 폴더/부모 폴더에서 같은 숫자/토큰 들어간 CSV 검색
    3순위: 그래도 없으면 None
    """
    mesh_path = Path(mesh_path)

    rid = extract_run_id_from_mesh(mesh_path)

    if rid is not None and str(rid) in idx:
        return idx[str(rid)]

    # 2) 근거리 폴더 휴리스틱
    candidates = []
    for scope in [mesh_path.parent, mesh_path.parent.parent]:
        if scope and scope.exists():
            for p in scope.glob("*.csv"):
                name = p.stem.lower()
                # 파일명 유사도 휴리스틱: 숫자 토큰/슬러그 공유 시
                if rid and (rid in name):
                    candidates.append(p)
            # 너무 많으면 break
            if candidates:
                break
    if candidates:
        # 가장 가까운 폴더의 첫 번째를 반환
        return sorted(candidates, key=lambda p: (p.parent != mesh_path.parent, p.name))[0]

    # 3) 인덱스에 unmatched가 있으면 하나라도 힌트 출력
    if fallbacks.get("__unmatched__"):
        # 디버그용: 가장 최근 CSV 하나 찍어주기
        return None
    return None


def _jsonable(obj):
    if isinstance(obj, Path):
        return str(obj)
    if isinstance(obj, (np.generic,)):  # np.int32, np.float32 등
        return obj.item()
    if isinstance(obj, (set, tuple)):
        return list(obj)
    if isinstance(obj, dict):
        return {str(k): _jsonable(v) for k, v in obj.items()}
    if isinstance(obj, (list,)):
        return [_jsonable(v) for v in obj]
    # 필요시 더 추가: e.g., torch.dtype, enum 등
    return obj

def safe_json_load(path):
    """Return obj or None if empty/corrupt."""
    try:
        with open(path, "rb") as f:
            raw = f.read()
        if not raw or not raw.strip():
            return None  # empty file
        # UTF-8 BOM 방지
        s = raw.decode("utf-8-sig")
        return json.loads(s)
    except Exception:
        return None  # corrupt

def safe_json_dump_atomic(obj, path):
    """Write JSON atomically to avoid partial writes."""
    d = os.path.dirname(path) or "."
    os.makedirs(d, exist_ok=True)
    fd, tmp = tempfile.mkstemp(prefix=".meta_tmp_", dir=d)
    try:
        with os.fdopen(fd, "w", encoding="utf-8") as f:
            json.dump(obj, f, ensure_ascii=False, sort_keys=True)
        os.replace(tmp, path)  # atomic on same filesystem
    except Exception:
        # 실패 시 임시파일 제거
        try: os.remove(tmp)
        except Exception: pass
        raise

def _extract_run_id_from_path(path_str: str, pattern=r"run[_\-]?(\d+)"):
    m = re.search(pattern, Path(path_str).stem)
    return m.group(1) if m else Path(path_str).stem

def _load_graph_level_target_from_csv(path_str: str, cfg: dict):
    """
    path_str: 현재 run의 메쉬 파일 경로
    cfg: options["GRAPH_TARGET"]
    return: np.ndarray(shape=[T], dtype=np.float32) or None
    """
    csv_dir   = Path(cfg["CSV_DIR"])
    run_id_rx = cfg.get("RUN_ID_REGEX", r"run[_\-]?(\d+)")
    csv_glob  = cfg.get("CSV_GLOB", "*.csv")
    cols      = cfg.get("TARGET_COLS", ["cd","cl"])
    agg       = cfg.get("AGG", "last").lower()
    fname_pat = cfg.get("FILENAME_MATCH", "{run_id}")


    run_id = _extract_run_id_from_path(path_str, run_id_rx)

    # 후보 CSV들 나열
    cands = sorted(csv_dir.glob(csv_glob))
    # 파일명에 run_id가 포함된 것 우선 필터
    cands = [p for p in cands if fname_pat.format(run_id=run_id) in p.stem] or cands
    if not cands:
        return None

    # 가장 그럴듯한 1개 선택(필요시 더 정교화 가능)
    csv_path = cands[0]
    try:
        df = pd.read_csv(csv_path)
    except Exception:
        return None

    if df.empty:
        return None

    # 여러 행이면 집계
    if agg == "last":
        row = df.iloc[-1]
    elif agg == "first":
        row = df.iloc[0]
    elif agg == "mean":
        row = df.mean(numeric_only=True)
    else:
        row = df.iloc[-1]

    out = []
    for c in cols:
        if c in row and np.isfinite(row[c]):
            out.append(float(row[c]))
        else:
            return None
    return np.asarray(out, dtype=np.float32)  # shape [T]

def canonical(name: str):
    return re.sub(r'[^a-z0-9]+', '', name.lower())

def _safe_is_numeric(arr):
    return isinstance(arr, np.ndarray) and arr.dtype.type in (np.float32, np.float64)

def _is_bad_array(arr):
    return not np.isfinite(arr).all()

def _maybe_reshape(arr):
    if arr.ndim == 1:
        return arr.reshape(-1, 1)
    return arr

def _zscore_inplace(arr):
    m = arr.mean(axis=0, keepdims=True)
    s = arr.std(axis=0, keepdims=True) + 1e-8
    return (arr - m) / s

def build_knn_edges(points, k=KNN_K):
    tree = cKDTree(points)
    # 내부 스레드 폭주 방지 위해 n_jobs 인자 제거 (단일스레드)
    dists, idx = tree.query(points, k=k+1)
    send = np.repeat(np.arange(points.shape[0]), k)
    recv = idx[:, 1:].reshape(-1)
    e = np.stack([send, recv], axis=0)
    rev = np.stack([recv, send], axis=1)
    all_e = np.concatenate([e, rev], axis=1)
    return np.unique(all_e, axis=1)

def _edge_from_faces_or_knn(surf_all_points, faces_arr, edge_mode="auto", k=KNN_K):
    if edge_mode == "knn":
        return build_knn_edges(surf_all_points, k=k)
    if edge_mode == "face" and (faces_arr is None or faces_arr.size == 0):
        return build_knn_edges(surf_all_points, k=k)

    if faces_arr is None or faces_arr.size == 0:
        return build_knn_edges(surf_all_points, k=k)

    try:
        tris = faces_arr.reshape(-1, 4)[:, 1:]
        e_pairs = np.vstack([tris[:, [0, 1]], tris[:, [1, 2]], tris[:, [2, 0]]])
        e_pairs_rev = e_pairs[:, [1, 0]]
        edge_index = np.unique(np.vstack([e_pairs, e_pairs_rev]), axis=0).T
        return edge_index
    except ValueError:
        return build_knn_edges(surf_all_points, k=k)

def find_target_name(mesh, target_field, auto_guess=True):
    point_fields = list(mesh.point_data.keys())
    cell_fields  = list(mesh.cell_data.keys())
    field_fields = list(mesh.field_data.keys())
    combined_set = set(point_fields) | set(cell_fields) | set(field_fields)

    if target_field in mesh.point_data:
        return target_field, 'point'
    if target_field in mesh.cell_data:
        return target_field, 'cell'

    canon_target = canonical(target_field)
    matches = [f for f in combined_set if canonical(f) == canon_target]
    if matches:
        t = matches[0]
        loc = 'point' if t in point_fields else ('cell' if t in cell_fields else 'field')
        return t, loc

    if auto_guess:
        heuristic = [f for f in combined_set if 'p' in f.lower() and ('coeff' in f.lower() or 'cp' in f.lower())]
        if heuristic:
            t = heuristic[0]
            loc = 'point' if t in point_fields else ('cell' if t in cell_fields else 'field')
            return t, loc

        close = difflib.get_close_matches(target_field, list(combined_set), n=1, cutoff=0.25)
        if close:
            t = close[0]
            loc = 'point' if t in point_fields else ('cell' if t in cell_fields else 'field')
            return t, loc

    return None, None


def _filter_run_dirs(run_dirs, run_range):
    if run_range is None:
        return run_dirs
    lo, hi = run_range
    kept = []
    for rd in run_dirs:
        try:
            n = int(rd.name.split("_")[1])
        except Exception:
            continue
        if lo <= n <= hi:
            kept.append(rd)
    return kept

def _fingerprint(mesh_files, extra: dict):
    entries = []
    for p in mesh_files:
        try:
            st = p.stat()
            entries.append([str(p), int(st.st_mtime), int(st.st_size)])
        except FileNotFoundError:
            entries.append([str(p), 0, 0])
    payload = {"files": entries, "extra": extra}

    payload_jsonable = _jsonable(payload)

    s = json.dumps(payload_jsonable, sort_keys=True).encode("utf-8")
    h = hashlib.sha1(s).hexdigest()
    return h, payload_jsonable


# Process One

In [5]:
# -------- 단일 파일 처리 (멀티프로세싱 대상) --------
def process_one(path_str: str, options: dict):
    import os
    import time
    import pyvista as pv

    pid = os.getpid()
    t0 = time.time()
    try:
        mesh = pv.read(path_str)
    except Exception as e:
        return None, ('read_error', os.path.basename(path_str), str(e))

    if mesh.n_points == 0:
        return None, ('empty', os.path.basename(path_str))

    t_read = time.time()

    # 표면 + 삼각화
    surf = mesh.extract_surface()
    if not getattr(surf, "is_all_triangles", False):
        try:
            surf = surf.triangulate()
        except Exception:
            pass
    t_surface = time.time()

    # cell->point
    if options["DO_CELL_TO_POINT"]:
        try:
            surf_all = surf.cell_data_to_point_data()
        except Exception:
            surf_all = surf
    else:
        surf_all = surf
    t_c2p = time.time()

    # 타깃 찾기
    target_name, source_loc = find_target_name(
        surf_all, options["TARGET_FIELD"], auto_guess=options["AUTO_GUESS_TARGET"]
    )
    if target_name is None:
        return None, ('no_target', os.path.basename(path_str),
                      list(surf_all.point_data.keys()), list(surf_all.cell_data.keys()))

    # 타깃 y
    if target_name in surf_all.point_data:
        y_raw = surf_all.point_data[target_name]
    else:
        if source_loc == 'point':
            try:
                sampled = surf_all.sample(mesh)
                y_raw = sampled.point_data[target_name]
            except Exception:
                return None, ('sample_fail', os.path.basename(path_str))
        elif source_loc == 'cell':
            return None, ('cell2point_fail', os.path.basename(path_str))
        else:
            return None, ('global_field', os.path.basename(path_str), target_name)

    pts = surf_all.points.astype(np.float32)


    # 특징 x
    feats = [pts]  # (N,3)
    if options["USE_NORMALS"] or options["DO_COMPUTE_NORMALS"]:
        if 'Normals' not in surf_all.point_data:
            try:
                surf_all.compute_normals(inplace=True, consistent_normals=True, auto_orient_normals=True)
            except TypeError:
                surf_all.compute_normals(inplace=True, auto_orient_normals=True)
        if 'Normals' in surf_all.point_data:
            nrm = surf_all.point_data['Normals']
            if _safe_is_numeric(nrm) and not _is_bad_array(nrm):
                feats.append(nrm.astype(np.float32))

    for name, arr in surf_all.point_data.items():
        if (not options["INCLUDE_TARGET_IN_X"]) and (canonical(name) == canonical(target_name)):
            continue
        if not isinstance(arr, np.ndarray):
            continue
        if arr.dtype.type not in (np.float32, np.float64):
            continue
        if arr.shape[0] != surf_all.n_points:
            continue
        if arr.ndim not in (1, 2):
            continue
        if arr.ndim == 2 and arr.shape[1] > options["MAX_FEATURE_DIM"]:
            continue
        if _is_bad_array(arr):
            continue
        feats.append(_maybe_reshape(arr.astype(np.float32)))

    if len(feats) == 0:
        return None, ('no_features', os.path.basename(path_str))

    x = np.concatenate(feats, axis=1)

    y = np.asarray(y_raw, dtype=np.float32).reshape(-1, 1)
    if y.shape[0] != surf_all.n_points:
        return None, ('len_mismatch', os.path.basename(path_str), y.shape[0], surf_all.n_points)

    # 정규화
    if options["NORMALIZE_X"]:
        x = _zscore_inplace(x)
    if options["NORMALIZE_TARGET"]:
        y = _zscore_inplace(y)


    # Graph level Y extraction (cd, cl)
    gt_cfg = options.get("GRAPH_TARGET", None)
    graph_target = None

    if gt_cfg:
        csv_dir   = gt_cfg.get("CSV_DIR", None)
        csv_glob  = gt_cfg.get("CSV_GLOB", "*.csv")
        runid_rx  = gt_cfg.get("RUN_ID_REGEX", [r"run[_\-]?(\d+)", r"_(\d+)$", r"(\d+)$"])
        target_cols = gt_cfg.get("TARGET_COLS", ["cd","cl"])
        agg = gt_cfg.get("AGG", "last").lower()
        ymode = gt_cfg.get("Y_MODE", "both").lower()

        if csv_dir and Path(csv_dir).exists():
            if not hasattr(process_one, "_CSV_INDEX_BUILT"):
                # 최초 1회 인덱스 구축 (함수 속성에 캐시)
                process_one._CSV_INDEX, process_one._CSV_FALLBACKS = _build_csv_index(csv_dir, csv_glob, runid_rx)
                process_one._CSV_INDEX_BUILT = True

            csv_idx   = process_one._CSV_INDEX
            csv_fbk   = process_one._CSV_FALLBACKS
            csv_path  = _find_csv_for_mesh(path_str, csv_idx, csv_fbk, runid_rx)

            if csv_path is None:
                print(f"[WARN] CSV not found for mesh: {path_str}")
            else:
                try:
                    df = pd.read_csv(csv_path)
                    if df.empty:
                        print(f"[WARN] CSV empty: {csv_path}")
                    else:
                        if agg == "last":
                            row = df.iloc[-1]
                        elif agg == "first":
                            row = df.iloc[0]
                        elif agg == "mean":
                            row = df.mean(numeric_only=True)
                        else:
                            row = df.iloc[-1]

                        vals = []
                        for c in target_cols:
                            # 대소문자/공백 무시 매칭
                            # ex) 'CD ', 'cd', 'Cd' 등도 잡도록
                            col_map = {str(k).strip().lower(): k for k in df.columns}
                            key = col_map.get(str(c).strip().lower(), None)
                            if key is None or (key not in row) or (not np.isfinite(row[key])):
                                vals = None
                                break
                            vals.append(float(row[key]))
                        if vals is not None:
                            graph_target = np.asarray(vals, dtype=np.float32)
                        else:
                            print(f"[WARN] target cols missing in CSV: {csv_path} -> {target_cols}")
                except Exception as e:
                    print(f"[WARN] CSV read fail: {csv_path} err={e}")
                    
    # Graph level Y (cd, cl) imposition
    Y_MODE = (gt_cfg.get("Y_MODE", "both").lower() if gt_cfg else "node_only")

    if Y_MODE == "graph_only" and (graph_target is not None):
        # 1) y를 그래프 레벨로만 사용
        y = graph_target  # np.float32 [T]
        y_is_graph_level = True
        # (선택) node-level 타겟은 완전히 생략

    elif Y_MODE == "both" and (graph_target is not None):
        # 2) 둘 다 유지: y_node는 기존대로, 그래프 타겟은 별도 필드로 Data에 넣음
        y_is_graph_level = False
        y_graph = graph_target  # np.float32 [T]

    else:
        # 3) 기본: node-only
        y_is_graph_level = False
        y_graph = None

    # 엣지
    faces_arr = getattr(surf_all, 'faces', None)
    edge_index = _edge_from_faces_or_knn(pts, faces_arr, edge_mode=options["EDGE_MODE"], k=options["KNN_K"])

    # 타이밍 로그
    t_edge = time.time()
    dt = {
        "read":      round(t_read - t0, 4),
        "surface":   round(t_surface - t_read, 4),
        "cell2pt":   round(t_c2p - t_surface, 4),
        "features+": round(t_edge - t_c2p, 4),
        "total":     round(t_edge - t0, 4)
    }
    print(f"[pid {pid}] {os.path.basename(path_str)} timings(sec): {dt}")

    data_kwargs = dict()
    data_kwargs["x"] = torch.from_numpy(x)
    data_kwargs["edge_index"] = torch.from_numpy(edge_index)

    if y_is_graph_level:
        # 그래프 레벨 타겟만 y로 사용
        data_kwargs["y"] = torch.from_numpy(graph_target)   # shape [T]
    else:
        # 기존 node-level y 유지
        data_kwargs["y"] = torch.from_numpy(y)
        if y_graph is not None:
            # 추가로 그래프 레벨 타겟을 별도 필드에 저장
            data_kwargs["y_graph"] = torch.from_numpy(y_graph)

    g = GeoData(**data_kwargs)

    return g, None





# Main

In [6]:
def main():
    # Define options directly within the function or pass them as arguments
    # For simplicity, using hardcoded defaults or values from the top-level constants
    DATA_ROOT = Path("d:\gnn\data") # Use the downloaded data path
    TARGET_FIELD = "static(p)_coeffMean"
    run_range = RUN_RANGE
    strict_boundary_only = STRICT_BOUNDARY_ONLY
    use_cache = CACHE_ENABLE
    max_workers = 60 # Or set to a reasonable default or user-defined value
    force_threadpool = False # Set to True if you want to force threadpool

    options = dict(
        DO_CELL_TO_POINT=DO_CELL_TO_POINT,
        USE_NORMALS=USE_NORMALS,
        DO_COMPUTE_NORMALS=DO_COMPUTE_NORMALS,
        INCLUDE_TARGET_IN_X=INCLUDE_TARGET_IN_X,
        NORMALIZE_X=NORMALIZE_X,
        NORMALIZE_TARGET=NORMALIZE_TARGET,
        MAX_FEATURE_DIM=MAX_FEATURE_DIM,
        EDGE_MODE=EDGE_MODE,
        KNN_K=KNN_K,
        TARGET_FIELD=TARGET_FIELD,
        AUTO_GUESS_TARGET=AUTO_GUESS_TARGET,
    )

    options.update({
    # 그래프 레벨 타겟 설정
    "GRAPH_TARGET": {                           # 없으면 건너뜀
        "CSV_DIR": DATA_ROOT,                   # 각 run마다 1개 CSV가 있는 폴더
        "RUN_ID_REGEX": r"run[_\-]?(\d+)",      # 파일명에서 run_id 추출
        "CSV_GLOB": "*.csv",                    # CSV 패턴
        "TARGET_COLS": ["cd", "cl"],                  # ["CD"] 또는 ["CD","CL"]
        "AGG": "last",                          # CSV가 여러 행이면: "last"|"mean"|"first"
        "Y_MODE": "both",                       # "both" | "graph_only" | "node_only"
        "FILENAME_MATCH": "{run_id}",           # (선택) CSV파일명이 run_id를 포함한다고 가정
    }
    })


    print("MAX_WORKERS =", max_workers)
    print("DATA_ROOT   =", DATA_ROOT)
    print("TARGET_FIELD=", TARGET_FIELD)
    print("EDGE_MODE   =", options['EDGE_MODE'])
    print("KNN_K       =", options['KNN_K'])
    print("RUN_RANGE   =", run_range)
    print("GRAPH_TARGET in options:", options.get("GRAPH_TARGET", None))


    # 1) run_* 디렉토리 스캔
    run_dirs = sorted([p for p in DATA_ROOT.glob('run_*') if p.is_dir()])
    if run_range is not None:
        run_dirs = _filter_run_dirs(run_dirs, run_range)

    print('Found run dirs in range:', len(run_dirs))
    if not run_dirs:
        print('DEBUG: No run_* directories in range under', DATA_ROOT.resolve())

    # 2) boundary_*.vtp 우선 수집
    mesh_files = []
    for rd in run_dirs:
        vtp_matches = sorted(rd.glob('boundary_*.vtp'))
        if vtp_matches:
            mesh_files.append(vtp_matches[0])
        elif not strict_boundary_only:
            alt = list(rd.glob('*.vtp'))
            if alt:
                mesh_files.append(alt[0])

    print('Meshes found:', len(mesh_files))
    if mesh_files[:5]:
        print('Sample mesh files:', [str(p) for p in mesh_files[:5]])
    if not mesh_files:
        print('DEBUG: No boundary_*.vtp files located. Check path / working directory.')

    # 3) 캐시 체크
    train_graphs, val_graphs = [], []
    cache_hit = False
    if use_cache and CACHE_META.exists() and CACHE_PATH.exists():
        with open(CACHE_META, "r", encoding="utf-8") as f:
            meta = safe_json_load(CACHE_META)
            if meta is None:
                print("[CACHE] meta corrupt/empty → rebuilding cache.")
                cache_hit = False
            else:
                cur_hash, cur_payload = _fingerprint(mesh_files, options)
                if meta.get("hash") == cur_hash:
                    try:
                        train_graphs, val_graphs = torch.load(CACHE_PATH)
                        cache_hit = True
                        print("[CACHE] hit ✓ → graphs loaded from cache.")
                    except Exception as e:
                        print("[CACHE] load failed:", e)
                        cache_hit = False
                else:
                    cache_hit = False

        cur_hash, cur_payload = _fingerprint(mesh_files, options)
        if meta.get("hash") == cur_hash:
            try:
                train_graphs, val_graphs = torch.load(CACHE_PATH)
                cache_hit = True
                print("[CACHE] hit ✓ → graphs loaded from cache.")
            except Exception as e:
                print("[CACHE] load failed:", e)

    if not cache_hit:
        # 4) 병렬 실행
        all_graphs = []
        skipped_missing_target, skipped_empty = [], []
        scan_reports = []
        errors_to_print = 3

        t0 = time.time()
        futures = {}
        if force_threadpool:
            print("[INFO] Force ThreadPoolExecutor")
            executor_ctx = ThreadPoolExecutor(max_workers=max_workers)
        else:
            executor_ctx = ProcessPoolExecutor(max_workers=max_workers)

        try:
            with executor_ctx as ex:
                for p in mesh_files:
                    futures[ex.submit(process_one, str(p), options)] = p
                for fut in tqdm(as_completed(futures), total=len(futures), desc=('Build graphs (thread)' if force_threadpool else 'Build graphs (proc)')):
                    g, info = fut.result()
                    if g is not None:
                        all_graphs.append(g)
                    else:
                        kind = info[0]
                        if kind in ('empty',):
                            skipped_empty.append(futures[fut])
                        else:
                            skipped_missing_target.append(futures[fut])
                            if errors_to_print > 0:
                                print('WARN:', info)
                                errors_to_print -= 1
        except Exception as e:
            if not force_threadpool:
                print("[WARN] ProcessPoolExecutor failed:", e)
                print(" → Falling back to ThreadPoolExecutor")
                all_graphs = []
                skipped_missing_target, skipped_empty = [], []
                errors_to_print = 3
                with ThreadPoolExecutor(max_workers=max_workers) as ex:
                    futures = {ex.submit(process_one, str(p), options): p for p in mesh_files}
                    for fut in tqdm(as_completed(futures), total=len(futures), desc='Build graphs (thread)'):
                        g, info = fut.result()
                        if g is not None:
                            all_graphs.append(g)
                        else:
                            kind = info[0]
                            if kind in ('empty',):
                                skipped_empty.append(futures[fut])
                            else:
                                skipped_missing_target.append(futures[fut])
                                if errors_to_print > 0:
                                    print('WARN:', info)
                                    errors_to_print -= 1
            else:
                raise
        t1 = time.time()
        print(f"Graph build elapsed: {t1 - t0:.2f}s")

        # 5) split
        random.shuffle(all_graphs)
        val_count = max(1, int(0.2 * len(all_graphs)))
        val_graphs = all_graphs[:val_count]
        train_graphs = all_graphs[val_count:]

        print(f'Graphs built: {len(all_graphs)}')
        print(f'Skipped (missing/invalid target): {len(skipped_missing_target)}')
        print(f'Skipped (empty meshes): {len(skipped_empty)}')
        if all_graphs:
            print('Train:', len(train_graphs), 'Val:', len(val_graphs))
            print('Feature dim:', train_graphs[0].x.size(1))

        # 6) 캐시 저장
        if use_cache and all_graphs:
            cur_hash, cur_payload = _fingerprint(mesh_files, options)
            torch.save((train_graphs, val_graphs), CACHE_PATH)
            with open(CACHE_META, "w", encoding="utf-8") as f:
                json.dump({"hash": cur_hash, "payload": cur_payload}, f, sort_keys=True, ensure_ascii=False)

            print("[CACHE] saved ✓")

    # 여기서부터 train_graphs, val_graphs 사용
    print("Done. You now have train_graphs / val_graphs in memory.")
    # 필요하면 이 자리에서 torch.save로 별도 저장도 가능
    # torch.save((train_graphs, val_graphs), "graphs_latest.pt")

# Removed the if __name__ == "__main__": block and argument parsing
from multiprocessing import freeze_support, set_start_method
freeze_support()  # Windows 안전
try:
    set_start_method("spawn")
except RuntimeError:
    # 이미 start method가 설정된 경우
    pass

# 캐시 파일 경로
CACHE_META = Path("graphs_cache.meta.json")
CACHE_PATH = Path("graphs_cache.pt")

# 메타 파일 삭제
if CACHE_META.exists():
    CACHE_META.unlink()
    print("Deleted", CACHE_META)

# 그래프 캐시 파일 삭제
if CACHE_PATH.exists():
    CACHE_PATH.unlink()
    print("Deleted", CACHE_PATH)

main()

MAX_WORKERS = 60
DATA_ROOT   = d:\gnn\data
TARGET_FIELD= static(p)_coeffMean
EDGE_MODE   = auto
KNN_K       = 8
RUN_RANGE   = (1, 101)
GRAPH_TARGET in options: {'CSV_DIR': WindowsPath('d:/gnn/data'), 'RUN_ID_REGEX': 'run[_\\-]?(\\d+)', 'CSV_GLOB': '*.csv', 'TARGET_COLS': ['cd', 'cl'], 'AGG': 'last', 'Y_MODE': 'both', 'FILENAME_MATCH': '{run_id}'}
Found run dirs in range: 102
Meshes found: 101
Sample mesh files: ['d:\\gnn\\data\\run_1\\boundary_1.vtp', 'd:\\gnn\\data\\run_10\\boundary_10.vtp', 'd:\\gnn\\data\\run_100\\boundary_100.vtp', 'd:\\gnn\\data\\run_101\\boundary_101.vtp', 'd:\\gnn\\data\\run_11\\boundary_11.vtp']


Build graphs (proc):   0%|          | 0/101 [00:01<?, ?it/s]


[WARN] ProcessPoolExecutor failed: A process in the process pool was terminated abruptly while the future was running or pending.
 → Falling back to ThreadPoolExecutor


Build graphs (thread):   1%|          | 1/101 [00:23<38:36, 23.16s/it]

[pid 25368] boundary_55.vtp timings(sec): {'read': 2.8275, 'surface': 0.3934, 'cell2pt': 0.1172, 'features+': 17.1148, 'total': 20.4529}


...

Build graphs (thread):  98%|█████████▊| 99/101 [25:47<00:42, 21.13s/it]

[pid 25368] boundary_59.vtp timings(sec): {'read': 5.4593, 'surface': 0.6845, 'cell2pt': 0.1811, 'features+': 1538.408, 'total': 1544.7328}
[pid 25368] boundary_38.vtp timings(sec): {'read': 5.3333, 'surface': 0.7892, 'cell2pt': 0.1716, 'features+': 1538.6568, 'total': 1544.9508}


Build graphs (thread): 100%|██████████| 101/101 [25:47<00:00, 15.33s/it]

[pid 25368] boundary_54.vtp timings(sec): {'read': 5.4248, 'surface': 0.7153, 'cell2pt': 0.1847, 'features+': 1538.8579, 'total': 1545.1827}
Graph build elapsed: 1550.08s
Graphs built: 101
Skipped (missing/invalid target): 0
Skipped (empty meshes): 0
Train: 81 Val: 20
Feature dim: 8


[CACHE] saved ✓
Done. You now have train_graphs / val_graphs in memory.


In [9]:
import torch

train_graphs, val_graphs = torch.load("graphs_cache.pt", weights_only=False)

print(len(train_graphs), len(val_graphs))
print(train_graphs[0].y_graph)   # 첫 그래프 구조 확인


81 20
tensor([0.3591, 0.4177])
